In [45]:
from numpy import *

def loadSimpleData():
    #构建data矩阵
    datMat = matrix([[ 1. ,  2.1],
        [ 2. ,  1.1],
        [ 1.3,  1. ],
        [ 1. ,  1. ],
        [ 2. ,  1. ]])
    #分类结果list
    classLabels = [1.0, 1.0, -1.0, -1.0, 1.0]
    return datMat,classLabels

def loadDataSet(file):
    numFeature = len(open(file).readline().split('\t'))
    dataMat = []
    labelMat = []
    fr = open(file)
    for line in fr.readlines():
        lineArr = []
        currLine = line.strip().split()
        for i in range(numFeature - 1):
            lineArr.append(float(currLine[i]))
        dataMat.append(lineArr)
        labelMat.append(float(currLine[-1]))
    return dataMat, labelMat

In [46]:
def stumpClassify(dataMatrix,dimen,threshVal,threshIneq):
    #initial all ones vector
    retArray = ones((shape(dataMatrix)[0],1))
    
    #update retArray according to threshIneq and theshVal
    if threshIneq == 'lt':
        retArray[dataMatrix[:,dimen] <= threshVal] = -1.0
    else:
        retArray[dataMatrix[:,dimen] > threshVal] = -1.0
        
    return retArray
    

def buildStump(dataArr,classLabels,D):
    dataMatrix = mat(dataArr)
    labelMat = mat(classLabels).T
    m,n = shape(dataMatrix)
    numSteps = 10.0
    bestStump = {}
    bestClassEst = mat(zeros((m,1)))
    minError = inf #init error sum, to +infinity
    
    #loop all features
    for i in range(n):
        rangeMin = dataMatrix[:,i].min()
        rangeMax = dataMatrix[:,i].max()
        stepSize = (rangeMax - rangeMin) / numSteps
        #loop over all range in current dimension
        for j in range(-1,int(numSteps)+1):
            for inequal in ['lt', 'gt']: #go over less than and greater than
                threshVal = (rangeMin + float(j) * stepSize)
                predictedVals = stumpClassify(dataMatrix,i,threshVal,inequal)#call stump classify with i, j, lessThan
                errArr = mat(ones((m,1)))
                #if predicted values equals to labels in labelMat, no error
                errArr[predictedVals == labelMat] = 0
                weightedError = D.T*errArr  #calc total error multiplied by D
                #print("split: dim %d, thresh %.2f, thresh ineqal: %s, the weighted error is %.3f" % 
                #     (i, threshVal, inequal, weightedError))
                if weightedError < minError:
                    minError = weightedError
                    bestClassEst = predictedVals.copy()
                    bestStump['dim'] = i
                    bestStump['thresh'] = threshVal
                    bestStump['ineq'] = inequal

    return bestStump,minError,bestClassEst

In [47]:
def adaBoostTrainDS(dataArr,classLabels,numIt=40):
    weakClassArr = []
    m = shape(dataArr)[0]
    D = mat(ones((m,1)) / m)
    aggClassEst = mat(zeros((m, 1)))
    for i in range(numIt):
        bestStump, error, classEst = buildStump(dataArr, classLabels, D)
        alpha = float(0.5 * log((1 - error) / max(error, 1e-16)))
        bestStump['alpha'] = alpha
        weakClassArr.append(bestStump)
        
        #update Dt+1
        expon = multiply(-1 * alpha * mat(classLabels).T, classEst)
        D = multiply(D, exp(expon))
        D = D / D.sum()
        
        #aggregate weak classifiers
        aggClassEst += alpha * classEst
        
        #calculate error, stop loop when error equals to zero
        aggErrors = multiply(sign(aggClassEst) != mat(classLabels).T, ones((m, 1)))
        errorRate = aggErrors.sum() / m
#         print("errorRate:", errorRate)
        if errorRate == 0.0:
            break
                      
    return weakClassArr, errorRate
            

In [48]:
dataMat, classLabels = loadSimpleData()
classifierArray,_ = adaBoostTrainDS(dataMat, classLabels, 9)

In [49]:
def adaClassify(dataToClass, classifierArr):
    dataMatrix = mat(dataToClass)
    m = dataMatrix.shape[0]
    aggClassEst = mat(zeros((m, 1)))
    for i in range(len(classifierArr)):
        classEst = stumpClassify(dataMatrix, 
                                 classifierArr[i]['dim'], 
                                 classifierArr[i]['thresh'], 
                                 classifierArr[i]['ineq'])
        #aggregate all weak classifier
        aggClassEst += classifierArr[i]['alpha']*classEst

    return sign(aggClassEst)

In [50]:
adaClassify([[5, 5], [0, 0]], classifierArray)

matrix([[ 1.],
        [-1.]])

In [51]:
datArr, labelArr = loadDataSet('horseColicTraining2.txt')
classifierArray,_ = adaBoostTrainDS(datArr, labelArr, 10)

testArr, testLabelArr = loadDataSet('horseColicTest2.txt')
prediction10 = adaClassify(testArr, classifierArray)

In [54]:
def verify(n):
    classifierArray,error = adaBoostTrainDS(datArr, labelArr, n)
    prediction = adaClassify(testArr, classifierArray)
    errArr = mat(ones((67, 1)))
    testErr = errArr[prediction != mat(testLabelArr).T].sum() / 67
    print("training error:", error)
    print("testErr %d: "% (n), testErr)

In [55]:
verify(10)
verify(50)
verify(100)
verify(500)
verify(1000)
verify(10000)

training error: 0.23076923076923078
testErr 10:  0.23880597014925373
training error: 0.18729096989966554
testErr 50:  0.208955223880597
training error: 0.19063545150501673
testErr 100:  0.22388059701492538
training error: 0.15719063545150502
testErr 500:  0.2537313432835821
training error: 0.14046822742474915
testErr 1000:  0.31343283582089554
training error: 0.11036789297658862
testErr 10000:  0.3283582089552239


In [ ]:
def plotROC(predStrengths, classLabels):
    import matplotlib.pyplot as plt
    curr = (1.0, 1.0)
    ySum = 0.0
    numPosClass = sum(array(classLabels)==1.0)
    yStep = 1 / float(numPosClass)
    xStep = 1 / float(len(classLabels) - numPosClass)
    storedIndices = predStrengths.argsort()
    fig = plt.figure()
    fig.clf()
    ax = plt.subplot(111)
    for index in sortedIndices.toList()[0]:
        if classLabels[index] == 1.0:
            delX = 0
            delY = yStep
        else:
            delX = xStep
            delY = 0
            ySum += cur[1]
        ax.plot([cur[0], cur[0]-delX], [cur[1], cur[1]-delY], c='b')
        cur = (cur[0]-delX, cur[1]-delY)
    ax.plot([0, 1],[0,1], 'b--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC curve for AdaBoost Horse Colic Detection System')
    ax.axis([0, 1, 0, 1])
    plt.show()
    print("the Area Under the Curve is :", ySum * xStep)
    
    